In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt


In [16]:
df = pd.read_stata(r"../data/final/panel_final_combined_data.dta")
df.columns

Index(['conglome_t1', 'vivienda_t1', 'hogar_t1', 'codperso_t1', 'ocupinf_t1',
       'ocupinf_t2', 'p207_t1', 'p208a_t1', 'p209_t1', 'p501_t1', 'p507_t1',
       'p510_t1', 'p510a1_t1', 'p510b_t1', 'p511a_t1', 'p512a_t1', 'p513a1_t1',
       'p513a2_t1', 'p513t_t1', 'p514_t1', 'p519_t1', 'p520_t1', 'p521a_t1',
       'p521c_t1', 'p524a1_t1', 'p524a2_t1', 'p528_t1', 'p530a_t1', 'p530b_t1',
       'p535_t1', 'p558c_t1', 'p558d_t1', 'p300a_t1', 'p301a_t1', 'p302_t1',
       'p307_t1', 'p313_t1', 'p314a_t1', 'p401f_t1', 'p401_t1', 'p4021_t1',
       'p4022_t1', 'p4023_t1', 'p4024_t1', 'p4025_t1', 'p4191_t1', 'p4192_t1',
       'p4193_t1', 'p4194_t1', 'p4195_t1', 'p4196_t1', 'p4197_t1', 'p4198_t1',
       'p101_t1', 'p102_t1', 'p103_t1', 'p103a_t1', 'p104_t1', 'p104a_t1',
       'p105a_t1', 'p105b_t1', 'p106_t1', 'p106b_t1', 'p107b1_t1', 'p110_t1',
       'p110a1_t1', 'p111a_t1', 'p112a_t1', 'p1142_t1', 'p1144_t1',
       'percepho_t1', 'mieperho_t1', 'ingbruhd_t1', 'ingindhd_t1',
       'i

In [17]:
# Descartamos las columnas que no son necesarias para el modelo
df = df.drop(columns=['conglome_t1', 'vivienda_t1','hogar_t1','codperso_t1','periodo','ocupinf_t1','ocupinf_t2','linea_t1'])

# Reordenar las columnas para que 'status_inf' sea la primera
df.insert(0, 'status_inf', df.pop('status_inf'))

# Calcular el tiempo total de trabajo en terminos anuales
# Crear la nueva columna 'p513_calculada_t1' que combine años y meses
df['p513_calculada_t1'] = df['p513a1_t1'] + df['p513a2_t1'] / 12
# Ver los primeros resultados para asegurarte de que la columna se calculó correctamente
df[['p513a1_t1', 'p513a2_t1', 'p513_calculada_t1']].head()
df = df.drop(columns=['p513a1_t1', 'p513a2_t1'])


In [4]:
# Lista de variables categóricas, excluyendo p513a1_t1 y p513a2_t1
categorical_columns_t1 = [
    'p207_t1', 'p209_t1', 'p501_t1', 'p507_t1', 'p510_t1', 'p510a1_t1', 'p510b_t1', 'p511a_t1', 'p512a_t1',
    'p514_t1', 'p519_t1', 'p521a_t1', 'p521c_t1', 'p524a2_t1', 'p528_t1', 'p530b_t1', 'p535_t1', 'p558c_t1',
    'p558d_t1', 'p300a_t1', 'p301a_t1', 'p302_t1', 'p307_t1', 'p313_t1', 'p314a_t1', 'p401f_t1', 'p401_t1',
    'p4021_t1', 'p4022_t1', 'p4023_t1', 'p4024_t1', 'p4025_t1', 'p4191_t1', 'p4192_t1', 'p4193_t1', 'p4194_t1',
    'p4195_t1', 'p4196_t1', 'p4197_t1', 'p4198_t1', 'p101_t1', 'p102_t1', 'p103_t1', 'p103a_t1', 'p104_t1',
    'p104a_t1', 'p105a_t1', 'p106b_t1', 'p107b1_t1', 'p110_t1', 'p110a1_t1', 'p111a_t1', 'p112a_t1', 'p1142_t1',
    'p1144_t1', 'estrsocial_t1', 'pobreza_t1'
]

# Filtrar el DataFrame para solo las variables categóricas
df_categoricas = df[categorical_columns_t1]

# Mostrar los valores únicos para cada columna categórica
for col in df_categoricas.columns:
    print(f"\nValores posibles para la columna '{col}':")
    print(df_categoricas[col].value_counts())


Valores posibles para la columna 'p207_t1':
p207_t1
hombre    12010
mujer      8031
Name: count, dtype: int64

Valores posibles para la columna 'p209_t1':
p209_t1
casado(a)        8261
conviviente      4857
soltero(a)       4212
separado(a)      2041
viudo(a)          421
divorciado(a)     249
Name: count, dtype: int64

Valores posibles para la columna 'p501_t1':
p501_t1
si    18893
no     1148
Name: count, dtype: int64

Valores posibles para la columna 'p507_t1':
p507_t1
empleado                    11001
obrero                       4214
trabajador independiente     3222
empleador o patrono          1524
trabajador del hogar           80
Name: count, dtype: int64

Valores posibles para la columna 'p510_t1':
p510_t1
empresa o patrono privado                                 7587
administración pública                                    6529
empresas especiales de servicios (service)                 487
fuerzas armadas, policía nacional del perú (militares)     448
empresa pública      

In [19]:
df.shape

(20041, 79)

In [12]:
import pandas as pd

# Suponiendo que tienes un DataFrame df con las columnas categóricas

# Lista de las columnas con solo dos categorías (basado en la revisión)
columns_two_categories = [
    'p514_t1', 'p519_t1', 'p521a_t1', 'p521c_t1', 'p524a2_t1', 'p528_t1', 'p530b_t1', 'p535_t1', 
    'p558d_t1', 'p4191_t1', 'p4192_t1', 'p4193_t1', 'p4194_t1', 'p4195_t1', 'p4196_t1', 'p4198_t1', 
    'p106b_t1', 'p107b1_t1', 'p110a1_t1', 'p1142_t1', 'p1144_t1'
]

# Convertir estas columnas en dummies (para columnas con 2 categorías)
for col in columns_two_categories:
    # Crear las columnas dummy para cada una de las variables
    df = pd.concat([df, pd.get_dummies(df[col], drop_first=True, prefix=col)], axis=1)
    df.drop(columns=[col], inplace=True)  # Eliminar la columna original

# Para las columnas con más de dos categorías, usar pd.get_dummies (One-Hot Encoding)
columns_more_than_two_categories = [
    'p207_t1', 'p209_t1', 'p507_t1', 'p510_t1', 'p510a1_t1', 'p511a_t1', 'p512a_t1', 'p558c_t1', 
    'p300a_t1', 'p301a_t1', 'p302_t1', 'p313_t1', 'p314a_t1', 'p401f_t1', 'p4021_t1', 
    'p4022_t1', 'p4023_t1', 'p4024_t1', 'p4025_t1'
]

# Crear dummies para las columnas con más de dos categorías
df = pd.get_dummies(df, columns=columns_more_than_two_categories, drop_first=False)

# Ahora tu DataFrame df tiene las columnas categóricas convertidas a dummies



In [22]:
print(df.values)

[['Se mantiene formal' 47 'si' ... False True False]
 ['Cae en informalidad' 57 'si' ... False True False]
 ['Se mantiene formal' 29 'si' ... False True False]
 ...
 ['Se mantiene formal' 29 'si' ... False False True]
 ['Se mantiene formal' 31 'si' ... False False True]
 ['Se mantiene formal' 30 'si' ... False False True]]
